# Moon Active Test

In [245]:
import pandas as pd
import random
import datetime
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists

## Creating the raw_data Table

In [246]:
# Using a generated users list (https://www.mockaroo.com/)
with open('users.txt') as usersfile:
    users = usersfile.read().split()

In [247]:
users

['632710dffc13ae5f50000007',
 '632710dffc13ae5f50000008',
 '632710dffc13ae5f50000009',
 '632710dffc13ae5f5000000a',
 '632710dffc13ae5f5000000b',
 '632710dffc13ae5f5000000c',
 '632710dffc13ae5f5000000d',
 '632710dffc13ae5f5000000e',
 '632710dffc13ae5f5000000f',
 '632710dffc13ae5f50000010',
 '632710dffc13ae5f50000011',
 '632710dffc13ae5f50000012',
 '632710dffc13ae5f50000013',
 '632710dffc13ae5f50000014',
 '632710dffc13ae5f50000015',
 '632710dffc13ae5f50000016',
 '632710dffc13ae5f50000017',
 '632710dffc13ae5f50000018',
 '632710dffc13ae5f50000019',
 '632710dffc13ae5f5000001a',
 '632710dffc13ae5f5000001b',
 '632710dffc13ae5f5000001c',
 '632710dffc13ae5f5000001d',
 '632710dffc13ae5f5000001e',
 '632710dffc13ae5f5000001f',
 '632710dffc13ae5f50000020',
 '632710dffc13ae5f50000021',
 '632710dffc13ae5f50000022',
 '632710dffc13ae5f50000023',
 '632710dffc13ae5f50000024',
 '632710dffc13ae5f50000025',
 '632710dffc13ae5f50000026',
 '632710dffc13ae5f50000027',
 '632710dffc13ae5f50000028',
 '632710dffc13

In [248]:
raw_data_table = pd.DataFrame(
    columns = ['activity_timestamp', 'user_id', 'session_id', 'activity', 'item_price', 'purchased_item', 'village'])
raw_data_table

,activity_timestamp,user_id,session_id,activity,item_price,purchased_item,village


In [249]:
# Create a dictionary with each user and the value of 1 for session start
users_sessions_dict = {i:1 for i in users}

In [250]:
# Appending a new row into raw_data
def add_row_to_raw_data(df):
    user = random.choice(users) # Current user_id

    current_session_id = users_sessions_dict[user] # Current session_id
    users_sessions_dict[user] = users_sessions_dict[user] + 1 # Update to the next session_id (+1)


    fake = Faker()
    current_timestamp = fake.date_time_between(start_date="-2y", end_date="now") # Current Timestamp

    activity = random.choice(['purchase completed', 'village_news_opened', 'village_completed']) # Current avticity

    if activity == 'purchase completed': 
        purchased_item = random.choice(['surprise_box', 'huge_spins_pack']) # purchased_item
        
        if purchased_item == 'surprise_box': # item_price
            item_price = 4
        else:
            item_price = 15
            
    else:
        purchased_item = None
        item_price = None

    village = random.randint(5, 250) # Current village

    new_df = df.append({'activity_timestamp':current_timestamp,
                           'user_id':user,
                           'session_id':current_session_id, 
                           'activity':activity,
                           'item_price':item_price, 
                           'purchased_item':purchased_item, 
                           'village':village},
                          ignore_index = True)
    
    return new_df

In [251]:
# Generating 10k records
for i in range(10000):
    raw_data_table = add_row_to_raw_data(raw_data_table)

In [252]:
raw_data_table.head()

,activity_timestamp,user_id,session_id,activity,item_price,purchased_item,village
0,2022-08-04 02:18:54,632710e2fc13ae5f50000310,1,purchase completed,15,huge_spins_pack,146
1,2021-05-24 05:06:02,632710e3fc13ae5f500003a7,1,purchase completed,4,surprise_box,155
2,2022-03-24 14:49:13,632710e1fc13ae5f5000016a,1,purchase completed,4,surprise_box,34
3,2021-06-27 03:06:13,632710e1fc13ae5f50000126,1,village_completed,None,None,187
4,2020-09-30 21:27:29,632710e2fc13ae5f50000317,1,village_news_opened,None,None,99


In [253]:
# raw_data_table.to_csv('raw_data.csv')

## Creating the coupons Table

In [254]:
coupons_table = pd.DataFrame(
    columns = ['coupon_id', 'start_at', 'ends_at', 'minimum_village', 'coupon_discount'])
coupons_table

,coupon_id,start_at,ends_at,minimum_village,coupon_discount


In [255]:
initial_timestamp = datetime.datetime.strptime('2020-01-01 00:00:01', '%Y-%m-%d %H:%M:%S') # Initial datetime
temp_timestamp = initial_timestamp

# Generating 1.5k records
for i in range(1, 1500):
    coupon_id = i # coupon_id
    
    start_at = temp_timestamp + datetime.timedelta(hours=random.randint(4, 8), 
                                                   minutes=random.randint(0, 60),
                                                   seconds=random.randint(0, 60)) # start_at
    if start_at > datetime.datetime.now():
        break
        
    ends_at = start_at + datetime.timedelta(hours=random.randint(1, 23)) # ends_at
    temp_timestamp = ends_at
    
    rnd_village = choice(range(0,205,5)) # minimum_village
    if rnd_village == 0:
        minimum_village = None
    else:
        minimum_village = rnd_village
    
    coupon_discount = choice([0.3, 0.5, 0.75]) # coupon_discount
    
    coupons_table = coupons_table.append({'coupon_id':coupon_id,
                                            'start_at':start_at,
                                            'ends_at':ends_at, 
                                            'minimum_village':minimum_village,
                                            'coupon_discount':coupon_discount},
                                            ignore_index = True)

In [256]:
coupons_table.head()

,coupon_id,start_at,ends_at,minimum_village,coupon_discount
0,1,2020-01-01 08:12:05,2020-01-01 22:12:05,95,0.75
1,2,2020-01-02 03:39:13,2020-01-02 05:39:13,25,0.30
2,3,2020-01-02 11:19:59,2020-01-03 02:19:59,110,0.50
3,4,2020-01-03 09:42:18,2020-01-03 19:42:18,25,0.50
4,5,2020-01-04 01:21:34,2020-01-04 02:21:34,5,0.50


In [257]:
# coupons_table.to_csv('coupons.csv')

## Connecting and Uploading Tables into MySQL Server

In [260]:
# Connecting MySQL
hostname="127.0.0.1"
dbname="moonactivetest"
username="root"
with open('pass.txt') as mypass:
    password = mypass.read()

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=hostname, db=dbname, user=username, pw=password))

In [261]:
coupons_table.to_sql('coupons', engine, index=False) # Uploading coupons table into MySQL's MoonActiveTest DB

In [262]:
raw_data_table.to_sql('raw_data', engine, index=False) # Uploading raw_data table

In [263]:
lifetime_table = pd.read_csv('lifetime.csv') # Uploading lifetime table (Generated via https://www.mockaroo.com/)

lifetime_table.to_sql('lifetime', engine, index=False)

## SQL Questions
*# Querying the server with SQLAlchemy*

### Question 1:
Find the country with the ghighest total deposits amount (off all users combined). Your answer should include the country name (column name: "country_name") and the total amount (column name: "lifetime_deposit_amount").

In [264]:
Query1 = '''
SELECT country_name, SUM(lifetime_deposit_amount) AS lifetime_deposit_amount 
FROM lifetime 
GROUP BY country_name 
ORDER BY lifetime_deposit_amount DESC 
LIMIT 1;
'''

with engine.connect() as con:

    rs = con.execute(Query1)
    df = pd.DataFrame(rs.fetchall(), columns = ['country_name', 'lifetime_deposit_amount'])
df

,country_name,lifetime_deposit_amount
0,China,10800


### Question 2:
Return the total number of sessions reported in raw_data (column name: "total_sessions"), and the number of sessions that included the activity: "village_completed" (column name: "village_comepleted_sessions"). Your answer should include a single row with two columns.

In [265]:
Query2 = '''
SELECT 
    (SELECT SUM(max_rd.max_session_id) AS total_sessions 
     FROM 
        (SELECT MAX(session_id) AS max_session_id 
         FROM raw_data 
         GROUP BY user_id) AS max_rd) AS total_sessions,
    (SELECT COUNT(session_id) 
     FROM raw_data 
     WHERE activity = 'village_completed') AS village_comepleted_sessions;
'''

with engine.connect() as con:

    rs = con.execute(Query2)
    df = pd.DataFrame(rs.fetchall(), columns = ['total_sessions', 'village_comepleted_sessions'])
df

,total_sessions,village_comepleted_sessions
0,10000,3347


### Question 3:
Return a list of all the paying users from the USA (users that had at least one purchase, column name: "user_id"), and the first item they purchased (column name: "purchased_item"). Order your answer by user id, descending.

In [266]:
Query3 = '''
SELECT firstitem_timestamp.user_id, purchased_item
FROM
    (SELECT lifetime.user_id, MIN(activity_timestamp) AS min_activity
     FROM lifetime
     INNER JOIN raw_data
         ON lifetime.user_id = raw_data.user_id
     WHERE country_name = 'United States' 
         AND purchased_item IS NOT NULL
     GROUP BY lifetime.user_id) AS firstitem_timestamp 
INNER JOIN raw_data 
    ON raw_data.user_id = firstitem_timestamp.user_id 
        AND firstitem_timestamp.min_activity = raw_data.activity_timestamp
ORDER BY firstitem_timestamp.user_id DESC;
'''

with engine.connect() as con:

    rs = con.execute(Query3)
    df = pd.DataFrame(rs.fetchall(), columns = ['user_id', 'purchased_item'])
df

,user_id,purchased_item
0,632710e3fc13ae5f500003ec,surprise_box
1,632710e3fc13ae5f500003cf,huge_spins_pack
2,632710e3fc13ae5f500003b7,huge_spins_pack
3,632710e3fc13ae5f500003a8,surprise_box
4,632710e3fc13ae5f50000397,huge_spins_pack
5,632710e2fc13ae5f500002ab,surprise_box
6,632710e2fc13ae5f5000026c,huge_spins_pack
7,632710e2fc13ae5f50000252,surprise_box
8,632710e2fc13ae5f50000237,huge_spins_pack
9,632710e1fc13ae5f500001b0,surprise_box


### Question 4:
For each coupon - report its ID (column name: "coupon_id"), the total price iOS users **actually** paid while it was active (column name: "total_deposits", rounded to 1 digit after the decimal), and the number of iOS users that paid while it was active (column name: "depositors"). Order your results by the amount paid, from the least to the most.

In [267]:
Query4 = '''
SELECT coupon_id, ROUND(SUM(item_price * (1-coupon_discount)), 1) AS total_depositors, COUNT(user_id) AS depositors
FROM
    (SELECT coupon_id, raw_data.user_id, activity_timestamp, activity, item_price, start_at, ends_at, coupon_discount, operating_system
    FROM raw_data
    INNER JOIN lifetime
        ON raw_data.user_id = lifetime.user_id
    INNER JOIN coupons
        ON raw_data.activity_timestamp BETWEEN coupons.start_at AND coupons.ends_at
    WHERE activity = 'purchase completed' and operating_system = "'iOS'") AS all_tables_relevant_columns
GROUP BY coupon_id
ORDER BY total_depositors;
'''

with engine.connect() as con:

    rs = con.execute(Query4)
    df = pd.DataFrame(rs.fetchall(), columns = ['coupon_id', 'total_depositors', 'depositors'])
df

,coupon_id,total_depositors,depositors
0,354,1.0,1
1,356,1.0,1
2,362,1.0,1
3,365,1.0,1
4,378,1.0,1
...,...,...,...
584,536,34.3,4
585,613,34.3,4
586,890,34.3,4
587,1226,34.3,4
